<a href="https://colab.research.google.com/github/edoob9/algorithm/blob/main/week5_siwoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문제

나는 새로 개편된 도시계획 및 기획부서의 담당자로 배정받았다. 오늘 아침 서울시장에 의해 강남구 서초1동 도시의 주택 지역에 주택과 병원이 위치할 수 있게 설계해달라는 공문이 내려왔다.이때 공문의 내용에는 추가로 주택과 경기도로 가는버스 정류장 사이의 최소 거리를 최소화할 수 있게 설계하라는 내용이 포함되어있었다.

( 단, 각 지역의 높이와 너비는 일정하다는 가정을 한다. 그리고 각 지역의 높이는 주어진다.)

기술 자문을 불러서 해당 문서에 대해서 설계하기 위한 조건들을 나열하라고 지시했다. 그 결과 문제의 조건은 다음과 같다.

1. 도시의 지도는 높이(height)와 너비(width)가 주어진 2차원 공간으로 표현됩니다.
2. 주택(house)은 이미 설정되어 있으며, 지도 내의 임의의 위치에 배치될 수 있습니다.
3. 병원(hospital)은 주택과 겹치지 않는 공간에 설치되어야 합니다.
4. 주택과 병원 사이의 거리는 맨해튼 거리로 측정됩니다.
5. 병원의 개수(num_hospitals)는 주어진 값으로 고정되어 있습니다.
6. 주택과 병원 사이의 최소 거리를 최소화하는 것이 목표입니다.

* fitness에 대한 내용 추가해야한다고 피드백을 받았음

## 결과
- cost
배치도 사진에 cost를 넣어서 한 눈에 볼 수 있게 제작해주세요.

### Reference

https://koreascience.kr/article/JAKO201710757619215.pdf

In [3]:
import random

class Space:
    def __init__(self, height, width, num_hospitals):
        self.height = height
        self.width = width
        self.num_hospitals = num_hospitals
        self.houses = set()
        self.hospitals = set()

    def add_house(self, row, col):
        self.houses.add((row, col))

    def available_space(self):
        candidates = set(
            (row, col)
            for row in range(self.height)
            for col in range(self.width)
        )
        for house in self.houses:
            candidates.remove(house)
        for hospital in self.hospitals:
            candidates.remove(hospital)
        return candidates

    def get_cost(self, hospitals):
        cost = 0
        for house in self.houses:
            cost += min(
                abs(house[0] - hospital[0]) + abs(house[1] - hospital[1])
                for hospital in hospitals
            )
        return cost

    def get_neighbors(self, row, col):
        candidates = [
            (row - 1, col),
            (row + 1, col),
            (row, col - 1),
            (row, col + 1)
        ]
        neighbors = []
        for r,c in candidates:
            if (r, c) in self.houses or (r, c) in self.hospitals:
                continue
            if 0 <= r < self.height and 0 <= c < self.width:
                neighbors.append((r, c))
        return neighbors

    def output_image(self, filename):
        from PIL import Image, ImageDraw, ImageFont
        cell_size = 100
        cell_border = 2
        cost_size = 40
        padding = 10

        img = Image.new("RGBA",
                        (self.width * cell_size, self.height * cell_size + cost_size + padding * 2),
                        "white"
                        )

        house = Image.open("/content/images/House.png").resize((cell_size, cell_size))
        hospital = Image.open("/content/images/Hospital.png").resize((cell_size, cell_size))
        draw = ImageDraw.Draw(img)

        for i in range(self.height):
            for j in range(self.width):
                rect = [
                    (j * cell_size + cell_border, i * cell_size + cell_border),
                    ((j + 1) * cell_size - cell_border, (i + 1) * cell_size - cell_border)
                ]
                draw.rectangle(rect, fill="black")

                if (i, j) in self.houses:
                    img.paste(house, rect[0], house)
                if (i, j) in self.hospitals:
                    img.paste(hospital, rect[0], hospital)

        draw.rectangle(
            (0, self.height * cell_size, self.width * cell_size,
             self.height * cell_size + cost_size + padding * 2),
            "black"
        )

        draw.text(
            (padding, self.height * cell_size + padding),
            f"Cost: {self.get_cost(self.hospitals)}",
            fill="white"
        )

        img.save(filename)

    def genetic_algorithm(self, population_size=100, generations=100, mutation_rate=0.1, image_prefix=None, log=False):
    # Initialize population
        population = self.initialize_population(population_size)

        for gen in range(generations):
            # Evaluate fitness
            fitness_scores = [(individual, self.get_cost(individual)) for individual in population]

            # Select parents
            parents = self.selection(fitness_scores, population_size)

            # Crossover
            offspring = self.crossover(parents)

            # Mutation
            offspring = [self.mutation(child, mutation_rate) for child in offspring]

            # Replace old population with offspring
            population = offspring

            # Find best individual in the population
            best_individual = min(fitness_scores, key=lambda x: x[1])[0]
            self.hospitals = set(best_individual)

            if log:
                print(f"Generation {gen + 1}, Best cost: {self.get_cost(best_individual)}")

            if image_prefix:
                self.output_image(f"{image_prefix}{str(gen + 1).zfill(3)}.png")

        return self.hospitals

    def initialize_population(self, population_size):
        population = []
        for _ in range(population_size):
            individual = set()
            for _ in range(self.num_hospitals):
                individual.add(random.choice(list(self.available_space())))
            population.append(individual)
        return population

    def selection(self, fitness_scores, population_size):
        # Tournament selection
        parents = []
        while len(parents) < population_size:
            tournament = random.sample(fitness_scores, k=5)
            winner = min(tournament, key=lambda x: x[1])[0]
            parents.append(winner)
        return parents


    def crossover(self, parents):
        # Single-point crossover
        offspring = []
        while len(offspring) < len(parents):
            parent1, parent2 = random.sample(parents, k=2)
            parent1_list = list(parent1)
            parent2_list = list(parent2)
            crossover_point = random.randint(1, self.num_hospitals - 1)
            child = set(parent1_list[:crossover_point] + parent2_list[crossover_point:])
            offspring.append(child)
        return offspring

    def mutation(self, individual, mutation_rate):
    # Flip mutation
        if random.random() < mutation_rate:
            mutated_gene = random.choice(list(self.available_space()))
            # 중복을 방지하기 위해 유전자가 개체에 없는 경우에만 추가합니다.
            if mutated_gene not in individual:
                individual.remove(random.choice(list(individual)))
                individual.add(mutated_gene)
        return individual

# Usage
s = Space(height=6, width=12, num_hospitals=3)
for i in range(5):
    s.add_house(random.randrange(s.height), random.randrange(s.width))

hospitals = s.genetic_algorithm(generations=10, image_prefix="hospitals", log=True)




Generation 1, Best cost: 7
Generation 2, Best cost: 7
Generation 3, Best cost: 5
Generation 4, Best cost: 5
Generation 5, Best cost: 5
Generation 6, Best cost: 5
Generation 7, Best cost: 5
Generation 8, Best cost: 5
Generation 9, Best cost: 5
Generation 10, Best cost: 5


In [4]:
s = Space(height=6, width=12, num_hospitals=3)
for i in range(5):
    s.add_house(random.randrange(s.height), random.randrange(s.width))

hospitals = s.genetic_algorithm(generations=10, image_prefix="hospitals", log=True)

Generation 1, Best cost: 8
Generation 2, Best cost: 8
Generation 3, Best cost: 8
Generation 4, Best cost: 8
Generation 5, Best cost: 7
Generation 6, Best cost: 7
Generation 7, Best cost: 7
Generation 8, Best cost: 7
Generation 9, Best cost: 7
Generation 10, Best cost: 7
